In [40]:
! pip install yfinance 

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import cvxpy as cp

(CVXPY) Mar 08 11:09:41 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 08 11:09:41 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [81]:
'''
amzn = yf.Ticker('AMZN')
pd.set_option('display.max_rows', None)
print(amzn.history(period="6mo"))
'''


pd.set_option('display.max_rows', None)
# download selected company historical data from Yahoo Finance
#companies = yf.download(['AAPL','AGG','DGRO','IJR','IXUS','NVDA','QQQ','SPY','TSLA','VTI','VUG'], period='6mo')
companies = yf.download(['MSFT','AAPL','BEP'],period='10y')
# Valid options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 
# 5y, 10y and ytd.

# Present the downloaded data
companies.hist


[*********************100%%**********************]  3 of 3 completed


<bound method hist_frame of Price        Adj Close                              Close             \
Ticker            AAPL        BEP        MSFT        AAPL        BEP   
Date                                                                   
2014-03-06   16.696165   9.065126   32.305595   18.955357  15.299760   
2014-03-07   16.686422   9.052499   32.093910   18.944286  15.278444   
2014-03-10   16.701517   8.970404   32.026161   18.961430  15.139888   
2014-03-11   16.864159   9.393504   32.195522   19.146070  15.853983   
2014-03-12   16.880508   9.036711   32.407227   19.164642  15.251799   
2014-03-13   16.693022   9.090389   32.085445   18.951786  15.342393   
2014-03-14   16.505537   9.096704   31.924541   18.738930  15.353051   
2014-03-17   16.570026   9.181955   32.220932   18.812143  15.496936   
2014-03-18   16.716614   9.090389   33.491127   18.978571  15.342393   
2014-03-19   16.712215   9.052499   33.254032   18.973572  15.278444   
2014-03-20   16.631674   8.910412   

In [82]:
# get all the close price data from the above table and print it out
companies_close_price = companies.get('Close')
print(companies_close_price)

Ticker            AAPL        BEP        MSFT
Date                                         
2014-03-06   18.955357  15.299760   38.150002
2014-03-07   18.944286  15.278444   37.900002
2014-03-10   18.961430  15.139888   37.820000
2014-03-11   19.146070  15.853983   38.020000
2014-03-12   19.164642  15.251799   38.270000
2014-03-13   18.951786  15.342393   37.889999
2014-03-14   18.738930  15.353051   37.700001
2014-03-17   18.812143  15.496936   38.049999
2014-03-18   18.978571  15.342393   39.549999
2014-03-19   18.973572  15.278444   39.270000
2014-03-20   18.882143  15.038636   40.330002
2014-03-21   19.031071  15.219824   40.160000
2014-03-24   19.256786  15.081268   40.500000
2014-03-25   19.463928  14.857447   40.340000
2014-03-26   19.277857  15.161204   39.790001
2014-03-27   19.195000  15.448974   39.360001
2014-03-28   19.173571  15.443645   40.299999
2014-03-31   19.169287  15.528910   40.990002
2014-04-01   19.344643  15.342393   41.419998
2014-04-02   19.376785  15.496936 

In [83]:
#data_close = pd.DataFrame(companies_close_price)
data_close = companies_close_price

In [84]:
# calculating the close price percentage change and round them to 3 decimal
data_close_pct_change = data_close.pct_change()
print(data_close_pct_change)

Ticker          AAPL       BEP      MSFT
Date                                    
2014-03-06       NaN       NaN       NaN
2014-03-07 -0.000584 -0.001393 -0.006553
2014-03-10  0.000905 -0.009069 -0.002111
2014-03-11  0.009738  0.047166  0.005288
2014-03-12  0.000970 -0.037983  0.006575
2014-03-13 -0.011107  0.005940 -0.009929
2014-03-14 -0.011231  0.000695 -0.005014
2014-03-17  0.003907  0.009372  0.009284
2014-03-18  0.008847 -0.009972  0.039422
2014-03-19 -0.000263 -0.004168 -0.007080
2014-03-20 -0.004819 -0.015696  0.026993
2014-03-21  0.007887  0.012048 -0.004215
2014-03-24  0.011860 -0.009104  0.008466
2014-03-25  0.010757 -0.014841 -0.003951
2014-03-26 -0.009560  0.020445 -0.013634
2014-03-27 -0.004298  0.018981 -0.010807
2014-03-28 -0.001116 -0.000345  0.023882
2014-03-31 -0.000223  0.005521  0.017122
2014-04-01  0.009148 -0.012011  0.010490
2014-04-02  0.001662  0.010073 -0.001690
2014-04-03 -0.006930 -0.001719 -0.008222
2014-04-04 -0.012936  0.003100 -0.027798
2014-04-07 -0.01

In [103]:
# find out the expected returned on each stock
means = data_close_pct_change.mean()
print(means)

Ticker
AAPL    0.001029
BEP     0.000320
MSFT    0.001082
dtype: float64


In [104]:
variance = data_close_pct_change.var()
print(variance)

Ticker
AAPL    0.000318
BEP     0.000326
MSFT    0.000290
dtype: float64


In [105]:
covariance = data_close_pct_change.cov()
print(covariance)

Ticker      AAPL       BEP      MSFT
Ticker                              
AAPL    0.000318  0.000110  0.000206
BEP     0.000110  0.000326  0.000102
MSFT    0.000206  0.000102  0.000290


In [111]:
num_vars = len(means)
x = cp.Variable(num_vars)
e = np.ones(num_vars)
Q = covariance
u = means.to_numpy()
R = [0.1]

objective = cp.Minimize(cp.quad_form(x,Q)- R*u@x)

constraints = [e @ x == 1, 
                x >= 0]

prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# Print result.
print("\nThe optimal return is", round(prob.value * 100,3), "%")
print("A solution x is")
print(np.round(x.value,3))


The optimal return is 0.011 %
A solution x is
[0.274 0.303 0.424]


In [91]:
num_vars = len(means)
x = cp.Variable(num_vars)
e = np.ones(num_vars)
Q = data_close_pct_change.cov()
u = means.to_numpy()
R = [0.08]

objective = cp.Maximize(u@x)

constraints = [e @ x == 1, 
               cp.quad_form(x,Q) <= R,
               x >= 0]

prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# Print result.
print("\nThe optimal return is", round(prob.value * 100,3), "%")
print("A solution x is")
print(np.round(x.value,3))



The optimal return is 0.108 %
A solution x is
[0. 0. 1.]


In [48]:
def cal_mean_and_var(stocks):
    companies_data = {}
    for s in stocks:
        # Download data from Yahoo Finance
        companies = yf.download(s, period='6mo')['Close']
        data = companies.copy()
        # Calculate the daily percentage change of return
        data_daily_pct_change = data.pct_change()
        # Calculate mean rate of return
        mean_rate_of_return = data_daily_pct_change.mean()
        # Calculate variance of return
        variance_of_return = data_daily_pct_change.var()
        # Store results in dictionary
        companies_data[s] = {'Mean Return': mean_rate_of_return, 'Variance Return': variance_of_return}
    # Convert dictionary to DataFrame
    data_pd = pd.DataFrame(companies_data)
    return data_pd

# Example list of stocks
stocks = ['AAPL','AGG','DGRO','IJR','IXUS','NVDA','QQQ','SPY','TSLA','VTI','VUG']
#stocks = ['AAPL']
# Run the fucntion
run_function = cal_mean_and_var(stocks)

print(run_function)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                     AAPL       AGG      DGRO       IJR      IXUS      NVDA  \
Mean Return     -0.000561  0.000226  0.000792  0.000747  0.000679  0.005455   
Variance Return  0.000133  0.000021  0.000047  0.000174  0.000070  0.000674   

                      QQQ       SPY      TSLA       VTI       VUG  
Mean Return      0.001319  0.001093 -0.002384  0.001089  0.001366  
Variance Return  0.000101  0.000058  0.000868  0.000063  0.000100  


In [112]:
'''
def cal_mean(stocks):
    companies_data = []
    for s in stocks:
        # Download data from Yahoo Finance
        companies = yf.download(s, period='6mo')['Close']
        data = companies.copy()
        # Calculate the daily percentage change of return
        data_daily_pct_change = data.pct_change()
        # Calculate mean rate of return
        mean_rate_of_return = round(data_daily_pct_change.mean(),6)
        # Append each return
        companies_data.append(mean_rate_of_return)
        
    return companies_data

def cal_var(stocks):
    companies_data = []
    for s in stocks:
        # Download data from Yahoo Finance
        companies = yf.download(s, period='6mo')['Close']
        data = companies.copy()
        # Calculate the daily percentage change of return
        data_daily_pct_change = data.pct_change()
         # Calculate variance of return
        variance_of_return = round(data_daily_pct_change.var(),6)
        # Append all var
        companies_data.append(variance_of_return)
    return companies_data
'''
def cal_mean(stocks):
    # Download data from Yahoo Finance
    companies = yf.download(stocks, period='6mo')['Close']
    data = companies.copy()
    # Calculate the daily percentage change of return        
    data_daily_pct_change = data.pct_change()
    # Calculate mean rate of return
    mean_rate_of_return = data_daily_pct_change.mean()
    return mean_rate_of_return


def cal_var(stocks):
# Download data from Yahoo Finance
    companies = yf.download(stocks, period='6mo')['Close']
    data = companies.copy()
    # Calculate the daily percentage change of return        
    data_daily_pct_change = data.pct_change()
    # Calculate mean rate of return
    variance_of_return = data_daily_pct_change.var()
    return variance_of_return

def cal_cov(stocks):
    companies = yf.download(stocks, period='6mo')['Close']
    data = companies.copy()
    data_daily_pct_change = data.pct_change()
    co_variance = data_daily_pct_change.cov()
    return co_variance


# Example list of stocks
stocks = ['AAPL','AGG','DGRO','IJR','IXUS','NVDA','QQQ','SPY','TSLA','VTI','VUG']
#stocks = ['AAPL']
# Run the fucntion
run_function = cal_mean(stocks)
run_function1 = cal_var(stocks)
run_function2 = cal_cov(stocks)

print(run_function)
print(run_function1)
#print(run_function2)

[*********************100%%**********************]  11 of 11 completed
[*********************100%%**********************]  11 of 11 completed
[*********************100%%**********************]  11 of 11 completed

Ticker
AAPL   -0.000557
AGG     0.000224
DGRO    0.000793
IJR     0.000764
IXUS    0.000682
NVDA    0.005424
QQQ     0.001317
SPY     0.001093
TSLA   -0.002344
VTI     0.001092
VUG     0.001365
dtype: float64
Ticker
AAPL    0.000133
AGG     0.000021
DGRO    0.000047
IJR     0.000174
IXUS    0.000070
NVDA    0.000672
QQQ     0.000101
SPY     0.000058
TSLA    0.000867
VTI     0.000063
VUG     0.000100
dtype: float64


In [2]:
C = np.array([[1, 2, 3],
              [4, 5, 6]])

# Find the shape of matrix C
n, m = C.shape

# Create a new matrix with the same shape as C filled with ones
ones_matrix = np.ones((n, m))

print("Shape of matrix C:", (n, m))
print("Matrix with ones of the same shape as C:")
print(ones_matrix)

Shape of matrix C: (2, 3)
Matrix with ones of the same shape as C:
[[1. 1. 1.]
 [1. 1. 1.]]


In [35]:
fq = 600
t = 10
p = 100000
R = 0.005
min_esg = 75
inv_type = 2
C = np.array([[4.95, (10**10)*p ,4.95],
              [4.95, (10**10)*p, 2.95],
              [0, (10**10)*p ,0],
              [4.95,(10**10)*p , 2.95]])

def cal_fee(matrix):
    n, m = matrix.shape
    one_matrix = np.ones((n,m))
    new_matrix = (((2+fq*t)/p) * matrix + one_matrix)**(1/t) - one_matrix
    return new_matrix

brokerA_i = np.array([[0.01, 0.02, 0.00],
                   [0.01, 0.02, 0.00],
                   [0.01, 0.02, 0.00],
                   [0.01, 0.02, 0.00]])
brokerB_i = np.array([[0.02, 0.01, 0.03],
                   [0.01, 0.01, 0.01],
                   [0.01, 0.00, 0.00],
                   [0.01, 0.00, 0.00]])
brokerC_i = np.array([[(4.95*((2+fq*t)/p)+1)**(1/t)-1, 10.00, (4.95*((2+fq*t)/p)+1)**(1/t)-1],
                   [(4.95*((2+fq*t)/p)+1)**(1/t)-1, 10.00, (4.95*((2+fq*t)/p)+1)**(1/t)-1],
                   [(0.00*((2+fq*t)/p)+1)**(1/t)-1, 10.00, (0.00*((2+fq*t)/p)+1)**(1/t)-1],
                   [(4.95*((2+fq*t)/p)+1)**(1/t)-1, 10.00, (4.95*((2+fq*t)/p)+1)**(1/t)-1]])


brokerA = brokerA_i[inv_type]
brokerB = brokerB_i[inv_type]
brokerC = brokerC_i[inv_type]

brokerMin = np.minimum.reduce([brokerA, brokerB, brokerC])
brokerMin = np.minimum.reduce([brokerA, brokerB])
print(brokerMin)

esg = np.array([75,80,65])
mu_1 = np.array([0.05, 0.04, 0.08])
Sigma = np.array([[0.0025, 0.0006, 0.002],
              [0.0006, 0.0016, 0.00128],
              [0.002, 0.00128, 0.0064]])
f = brokerMin
mu = mu_1 - f
n = len(mu)
x = cp.Variable(n)


objective = cp.Maximize(mu.T @ x -0.00)
constraints = [cp.sum(x) == 1, 
               cp.quad_form(x, Sigma) <= R,
               esg.T @ x >= min_esg,
               x >= 0]

problem = cp.Problem(objective, constraints)

problem.solve()

optimal_value = problem.value
optimal_solution = x.value
print("Optimal value:", optimal_value)
print("Optimal solution:", optimal_solution)

[0.01 0.   0.  ]
Optimal value: 0.053333333320938925
Optimal solution: [8.25958022e-10 6.66666666e-01 3.33333333e-01]


In [36]:
print(cal_fee(C))

[[2.63543148e-02 2.28687458e+01 2.63543148e-02]
 [2.63543148e-02 2.28687458e+01 1.64354963e-02]
 [0.00000000e+00 2.28687458e+01 0.00000000e+00]
 [2.63543148e-02 2.28687458e+01 1.64354963e-02]]
